In [87]:
import bs4 as bs
import pickle
import requests
import pandas as pd
import yfinance as yf
import math
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font

In [2]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

# Call the function to get the tickers
tickers_list = save_sp500_tickers()

In [3]:
# Create a DataFrame from the list of tickers
tickers_df = pd.DataFrame({'Ticker': tickers_list})

# Save the DataFrame to a new CSV file
tickers_df.to_csv('sp500_tickers.csv', index=False)

print("S&P 500 tickers saved to sp500_tickers.csv.")

S&P 500 tickers saved to sp500_tickers.csv.


In [88]:
# Removing\n at the end of each stock
tickers_df['Ticker'] = tickers_df['Ticker'].str.replace('\n', '')

tickers_list1 = tickers_df['Ticker'].tolist()

In [5]:
tickers_list1 = [ticker.replace("BRK.B", "BRK-B").replace("BF.B", "BF-B") for ticker in tickers_list1]

In [16]:
# Initialize empty lists to store data
tickers = []
open_prices = []
market_caps = []

# Function to fetch market cap for a given ticker using Yahoo Finance API
def get_market_cap(ticker):
    try:
        stock_data = yf.download(ticker, period='1d')
        if len(stock_data) > 0:
            latest_data = stock_data.iloc[-1]
            open_price = latest_data['Open']
            volume = latest_data['Volume']
            market_cap = open_price * volume
            return open_price, market_cap
        else:
            return None, None
    except:
        return None, None

# Use the function to fetch data for each ticker
for ticker in tickers_list1:
    open_price, market_cap = get_market_cap(ticker)
    if open_price is not None and market_cap is not None:
        tickers.append(ticker)
        open_prices.append(open_price)
        market_caps.append(market_cap)

# Create a DataFrame with the obtained data
data = {
    'Ticker': tickers,
    'Open Price': open_prices,
    'Market Cap': market_caps
}
tickers_df = pd.DataFrame(data)

# Display the final DataFrame
print(tickers_df)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [33]:
tickers_df["Number of Shares to Buy"] = None
tickers_df

,Ticker,Open Price,Market Cap,Number of Shares to Buy
0,MMM,104.169998,2.602896e+08,None
1,AOS,76.260002,5.702723e+07,None
2,ABT,112.849998,7.970257e+08,None
3,ABBV,142.199997,8.040414e+08,None
4,ACN,313.190002,1.022785e+09,None
...,...,...,...,...
498,YUM,136.419998,1.755316e+08,None
499,ZBRA,308.019989,6.625510e+07,None
500,ZBH,142.539993,1.601009e+08,None
501,ZION,38.070000,2.199647e+08,None


In [28]:
portfolio_size = input("Enter the value of your portfolio")

try:
    val = float(portfolio_size)
except ValueError:
    print("Thats not a number \n Please try again")
    portfolio_size = input("Enter the value of your portfolio")
    val = float(portfolio_size)

In [74]:
position_size = val/len(tickers_df)
#print(position_size)

for i in range(0,len(tickers_df.index)):
    tickers_df.loc[i,"Number of Shares to Buy"] = math.floor(position_size/tickers_df.loc[i,"Open Price"])

tickers_df

,Ticker,Open Price,Market Cap,Number of Shares to Buy
0,MMM,104.169998,2.602896e+08,190
1,AOS,76.260002,5.702723e+07,260
2,ABT,112.849998,7.970257e+08,176
3,ABBV,142.199997,8.040414e+08,139
4,ACN,313.190002,1.022785e+09,63
...,...,...,...,...
498,YUM,136.419998,1.755316e+08,145
499,ZBRA,308.019989,6.625510e+07,64
500,ZBH,142.539993,1.601009e+08,139
501,ZION,38.070000,2.199647e+08,522


In [86]:
# Define the Excel file name
excel_file_name = 'Recommended Trades.xlsx'

# Create a Pandas Excel writer using the openpyxl engine
writer = pd.ExcelWriter(excel_file_name, engine='openpyxl')

# Write the DataFrame to the Excel file
tickers_df.to_excel(writer, index=False)
# Get the workbook and worksheet objects
workbook1  = writer.book
worksheet = writer.sheets['Sheet1']

# Create cell formatting
header_fill = PatternFill(start_color='0A0A23', end_color='0000FF', fill_type='solid')
header_font = Font(color='FFFFFF', bold=True)
currency_format = '[$$-409]#,##0.00'

# Format the header row (first row)
for col_num, column_letter in enumerate(worksheet.iter_cols(min_col=1, max_col=len(tickers_df.columns)), 1):  # Adjust the index to start from 1
    for cell in column_letter:
        cell.fill = header_fill
        cell.font = header_font

# Format the second column as currency
for cell in worksheet['B'][1:]:  # Starting from the second row
    cell.number_format = currency_format

# Save the Excel file
writer.save()